# Model Accuracy

Up until now, I've been using mostly qualitative, visual assessment to determine how well a model performed. I want to create a better metric. I'm thinking that a good metric will be the percentage of pixels correctly labelled; let's see how well it works.

In [1]:
import torch

import monai
from monai.transforms import (
    Compose,
    LoadNiftid,
    NormalizeIntensityd,
    AddChanneld,
    ToTensord,
    CenterSpatialCropd,
)

import numpy as np

In [2]:
def get_model_accuracy(model):
    # Get data from test file doc
    data_dir = 'data/'
    test_imgs = []
    with open(data_dir + 'test_imgs.txt', 'r') as f:
        test_imgs = [image.rstrip() for image in f.readlines()]

    test_masks = []
    with open(data_dir + 'test_masks.txt', 'r') as f:
        test_masks = [mask.rstrip() for mask in f.readlines()]

    test_dicts = [{'image': image, 'mask': mask} for (image, mask) in zip(test_imgs, test_masks)]
    
    data_keys = ['image', 'mask']

    test_transforms = Compose(
        [
            LoadNiftid(keys=data_keys),
            AddChanneld(keys=data_keys),
            NormalizeIntensityd(keys="image"),
            CenterSpatialCropd(
                keys=data_keys,
                roi_size=(256, 256, 16),
            ),
            ToTensord(keys=data_keys),
        ]
    )

    test_dataset = monai.data.Dataset(
        data=test_dicts,
        transform=test_transforms,
    )
        
    accuracies = np.array([])
    for sample in test_dataset:
        test_image = sample['image'].unsqueeze(0)
        test_mask = model(test_image)
        test_mask = test_mask.argmax(1).detach()
        
        accuracy = 1 - torch.sum(torch.abs(sample['mask'] - test_mask)) / (256 * 256 * 16)
        accuracies = np.append(accuracies, accuracy)
        
    avg_accuracy = np.average(accuracies)
    median_accuracy = np.median(accuracies)
    return {
        'average': avg_accuracy,
        'median': median_accuracy
    }

In [3]:
import glob
import CustomModels

In [4]:
def print_acc_metrics(model_class, checkpoint_path):
    model = model_class.load_from_checkpoint(checkpoint_path)
    print(model.hparams.name)
    accuracy = get_model_accuracy(model)
    print('Average Accuracy: ', accuracy['average'])
    print('Median Accuracy: ', accuracy['median'])

## 3D UNet

In [5]:
print_acc_metrics(CustomModels.UNet_3D, 'models/7-15-2020_3dremastered/_ckpt_epoch_99.ckpt')

7-8-2020_3dremastered
Average Accuracy:  0.978900655110677
Median Accuracy:  0.9790596961975098


## Dice + Focal Loss

In [6]:
print_acc_metrics(CustomModels.UNet_DF, 'models/7-8-2020_dicefocal/_ckpt_epoch_119.ckpt')

7-8-2020_dicefocal
Average Accuracy:  0.9767536799112956
Median Accuracy:  0.9764394760131836


## MaskGAN

In [7]:
print_acc_metrics(CustomModels.MaskGAN, 'models/7-16-2020_MaskGAN2/_ckpt_epoch_234.ckpt')

7-16-2020_MaskGAN2
Average Accuracy:  0.9758198738098145
Median Accuracy:  0.9759783744812012
